# Makayla McKibben
## DSC 550
## Ex. 3.2

In [254]:
# Import relevant libraries
import pandas as pd
import numpy as np
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\makay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\makay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [255]:
from nltk.stem.porter import PorterStemmer

In [256]:
from nltk.corpus import stopwords

In [257]:
from nltk.tokenize import word_tokenize

In [258]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [259]:
from textblob import TextBlob

In [260]:
from sklearn.feature_extraction import DictVectorizer

In [261]:
from sklearn.feature_extraction.text import CountVectorizer

In [262]:
from sklearn.feature_extraction.text import TfidfVectorizer

## Part 1 Using the TextBlob Sentiment Analyzer

In [264]:
# Read the file into a dataframe
movie_reviews = pd.read_csv('labeledTrainData.tsv', sep = '\t')

In [265]:
# Verify it's read in as a dataframe
type(movie_reviews)

pandas.core.frame.DataFrame

In [266]:
# Look at the structure of the df
movie_reviews.head(5)
movie_reviews.head(-5)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24990,2034_9,1,I've never been huge on IMAX films. They're co...
24991,9416_3,0,Steve McQueen has certainly a lot of loyal fan...
24992,10994_1,0,Sometimes you wonder how some people get fundi...
24993,10957_3,0,"I am a student of film, and have been for seve..."


In [267]:
# Check the rows and columns length
movie_reviews.shape

(25000, 3)

In [268]:
# Find number of reviews that are negative
len(movie_reviews[(movie_reviews['sentiment'] == 0)])

12500

In [269]:
# Find number of reviews that are positive
len(movie_reviews[(movie_reviews['sentiment'] == 1)])

12500

In [270]:
# Use TextBlob to see if the first entry is negative or positive
first_review = movie_reviews['review'][0]
first_review = TextBlob(first_review)
first_review.sentiment

Sentiment(polarity=0.001276742581090417, subjectivity=0.6067460317460317)

In [271]:
# Define the function that will check the sentiment of each review
def blob_maker(review):
    review = TextBlob(review)
    review_sentiment = review.sentiment
    return review_sentiment

In [272]:
# Apply the function that sets each review to raw numbers and place it into a new column
movie_reviews['sentiment_textblob'] = movie_reviews['review'].apply(blob_maker)

In [273]:
# Look at the output in the new column
movie_reviews['sentiment_textblob']

0          (0.001276742581090417, 0.6067460317460317)
1            (0.2563492063492064, 0.5311111111111111)
2          (-0.05394123606889564, 0.5629331306990881)
3           (0.1347530864197531, 0.49290123456790125)
4        (-0.024841720779220786, 0.45981782106782115)
                             ...                     
24995       (0.10208333333333333, 0.5428571428571428)
24996       (0.09081262939958591, 0.4623706004140787)
24997      (0.14525641025641026, 0.48410256410256414)
24998                  (0.065625, 0.5045138888888889)
24999        (0.2392948717948718, 0.7358974358974358)
Name: sentiment_textblob, Length: 25000, dtype: object

In [274]:
# look at the whole df
movie_reviews.head()

,id,sentiment,review,sentiment_textblob
0,5814_8,1,With all this stuff going down at the moment w...,"(0.001276742581090417, 0.6067460317460317)"
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","(0.2563492063492064, 0.5311111111111111)"
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,"(-0.05394123606889564, 0.5629331306990881)"
3,3630_4,0,It must be assumed that those who praised this...,"(0.1347530864197531, 0.49290123456790125)"
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,"(-0.024841720779220786, 0.45981782106782115)"


In [275]:
# Define the function that classifies each review as negative or positive
def determine_sentiment(blob_output):
    sentiment = blob_output[0]
    if sentiment >= 0:
        sentiment = 'Positive'
        pass
    else:
        sentiment = 'Negative'
        pass
    return sentiment

In [276]:
# Apply the function that determines if a review is negative or positive and assign it to a new column
movie_reviews['blob_sentiment'] = movie_reviews['sentiment_textblob'].apply(determine_sentiment)

In [277]:
# Check the output for the new column
movie_reviews['blob_sentiment']

0        Positive
1        Positive
2        Negative
3        Positive
4        Negative
           ...   
24995    Positive
24996    Positive
24997    Positive
24998    Positive
24999    Positive
Name: blob_sentiment, Length: 25000, dtype: object

In [278]:
# Check how many reviews are positive
len(movie_reviews[(movie_reviews['blob_sentiment'] == 'Positive')])

19017

In [279]:
# Check how many reviews are negative
len(movie_reviews[(movie_reviews['blob_sentiment'] == 'Negative')])

5983

In [280]:
# Find the number of matches between the df assigned sentiment and the text blob sentiment
text_blob_accuracy_pos = movie_reviews[(movie_reviews['blob_sentiment'] == 'Positive') & (movie_reviews['sentiment'] == 1)]

In [281]:
# Find the total number of positive matches
positive_matches = len(text_blob_accuracy_pos)

In [282]:
# Find the number of matches between the df assigned sentiment and the text blob sentiment
text_blob_accuracy_neg = movie_reviews[(movie_reviews['blob_sentiment'] == 'Negative') & (movie_reviews['sentiment'] == 0)]

In [283]:
# Find the total number of negative matches
negative_matches = len(text_blob_accuracy_neg)

In [284]:
# Calculate and print accuracy of TextBlob sentiment analysis
total_acc = ((positive_matches + negative_matches) / len(movie_reviews)) * 100
print('Overall accuracy of the textblob sentiment analysis is: ', total_acc, '%')

Overall accuracy of the textblob sentiment analysis is:  68.524 %


Logic would dictate that we should correctly predict 50% of the sentiment results through random guesses. The textblob model correctly predicted 68.524% which is a substantial improvement.

## Part 2. Prepping Text for a Custom Model

In [287]:
# Define the function to make all text lowercase
def lowercase(text):
    return text.lower()

In [288]:
# Apply the lowercase function
movie_reviews['processed_text'] = movie_reviews['review'].apply(lowercase)

In [289]:
# Define the function to remove punctuation
def remove_punc(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [290]:
# Apply function that removes punctuation
movie_reviews['processed_text'] = movie_reviews['processed_text'].apply(remove_punc)

In [291]:
# Set up the stop words values
stop_words = stopwords.words('english')

In [292]:
# Define the function that removes the stop words from the text
def remove_stop_words(text):
    tokenized_words = word_tokenize(text)
    no_stops = [word for word in tokenized_words if word not in stop_words]
    return no_stops

In [293]:
# Remove the stop words
movie_reviews['processed_text'] = movie_reviews['processed_text'].apply(remove_stop_words)

In [294]:
# Look at the reviews
movie_reviews['processed_text'].head()

0    [stuff, going, moment, mj, ive, started, liste...
1    [classic, war, worlds, timothy, hines, enterta...
2    [film, starts, manager, nicholas, bell, giving...
3    [must, assumed, praised, film, greatest, filme...
4    [superbly, trashy, wondrously, unpretentious, ...
Name: processed_text, dtype: object

In [295]:
# Set up stem function
porter = PorterStemmer()

In [296]:
# Define stemming function
def stems(list_of_words):
    stemmed = [porter.stem(word) for word in list_of_words]
    return stemmed

In [297]:
# Apply stem function
movie_reviews['stemmed_text'] = movie_reviews['processed_text'].apply(stems)

In [298]:
# Look at the processed reviews
movie_reviews['stemmed_text'].head()

0    [stuff, go, moment, mj, ive, start, listen, mu...
1    [classic, war, world, timothi, hine, entertain...
2    [film, start, manag, nichola, bell, give, welc...
3    [must, assum, prais, film, greatest, film, ope...
4    [superbl, trashi, wondrous, unpretenti, 80, ex...
Name: stemmed_text, dtype: object

In [299]:
# Join reviews into a single string for each row
movie_reviews['stemmed_text'] = movie_reviews['stemmed_text'].apply(lambda lst: " ". join(lst))

In [300]:
# Look at reviews to verify they've been stemmed
movie_reviews['stemmed_text']

0        stuff go moment mj ive start listen music watc...
1        classic war world timothi hine entertain film ...
2        film start manag nichola bell give welcom inve...
3        must assum prais film greatest film opera ever...
4        superbl trashi wondrous unpretenti 80 exploit ...
                               ...                        
24995    seem like consider gone imdb review film went ...
24996    dont believ made film complet unnecessari firs...
24997    guy loser cant get girl need build pick strong...
24998    30 minut documentari buñuel made earli 1930 on...
24999    saw movi child broke heart stori unfinish end ...
Name: stemmed_text, Length: 25000, dtype: object

In [301]:
# Define the function that sets up the bag of words matrix
def get_the_bag(text):
    count = CountVectorizer()
    bag_of_words = count.fit_transform(text)
    return bag_of_words

In [302]:
# Convert string to text for bag of words to work
movie_reviews['polished_text'] = movie_reviews['stemmed_text'].apply(lambda x: [x])

In [303]:
# Apply the bag of words function
bag_of_words = movie_reviews['polished_text'].apply(get_the_bag)

In [304]:
# Check the shape of the bag of words
bag_of_words.shape

(25000,)

In [305]:
# Look at the review format
bag_of_words[0]

<1x161 sparse matrix of type '<class 'numpy.int64'>'
	with 161 stored elements in Compressed Sparse Row format>

In [306]:
# Define the term frequency-inverse document frequency (tf-idf) function
def tfidf(text):
    tfidf_func = TfidfVectorizer()
    term_freq = tfidf_func.fit_transform(text)
    return term_freq

In [307]:
# Apply the tfidf function
movie_reviews['tfidf'] = movie_reviews['polished_text'].apply(tfidf)

In [308]:
# Check that the tfidf shape matches the bag_of_words shape
movie_reviews['tfidf'][0]

<1x161 sparse matrix of type '<class 'numpy.float64'>'
	with 161 stored elements in Compressed Sparse Row format>